# 🧪 Codelab: Deutsch-Jozsa Algorithm

| Metadata | Value |
|----------|-------|
| **Algorithm** | Deutsch-Jozsa |
| **Difficulty** | 🟡 Intermediate |
| **Time** | 90-120 minutes |
| **Prerequisites** | Qubits, Hadamard gates, CNOT, Measurement |
| **Qiskit Version** | 2.x |

---

## Learning Objectives

By the end of this codelab, you will be able to:

1. ✅ Implement constant and balanced oracles
2. ✅ Construct the complete Deutsch-Jozsa circuit
3. ✅ Understand and visualize phase kickback
4. ✅ Run experiments on simulators and analyze results
5. ✅ Understand error effects on real hardware

## Section 1: Environment Setup & Version Check

In [ ]:
# Required imports
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, List, Tuple, Optional

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_aer import AerSimulator

# Version check - Qiskit 2.x required
import qiskit
version = qiskit.__version__
major_version = int(version.split('.')[0])
assert major_version >= 1, f"Qiskit 2.x required, found {version}"
print(f"✓ Qiskit version: {version}")
print(f"✓ NumPy version: {np.__version__}")
print("✓ All imports successful!")

## Section 2: Theory Recap

### The Problem

Given a Boolean function $f: \{0,1\}^n \rightarrow \{0,1\}$ with the **promise** that it is either:
- **Constant**: $f(x) = c$ for all $x$ (same output for every input)
- **Balanced**: $f(x) = 0$ for exactly half of inputs, $f(x) = 1$ for the other half

**Goal**: Determine if $f$ is constant or balanced.

### Classical vs Quantum Complexity

| Approach | Queries Needed |
|----------|---------------|
| Classical Deterministic | $2^{n-1} + 1$ (worst case) |
| Classical Randomized | $O(1)$ (high probability) |
| **Quantum (DJ)** | **1** (100% certainty) |

### The Key Insight: Phase Kickback

The oracle $U_f$ transforms: $|x\rangle|y\rangle \rightarrow |x\rangle|y \oplus f(x)\rangle$

When the ancilla is in $|-\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$:

$$|x\rangle|-\rangle \xrightarrow{U_f} (-1)^{f(x)}|x\rangle|-\rangle$$

The function value becomes a **phase** on the input register!

## Section 3: Basic Implementation - Constant Oracle

In [ ]:
def create_constant_oracle(n: int, output_value: int = 0) -> QuantumCircuit:
    """
    Create a constant oracle: f(x) = output_value for all x.
    
    Args:
        n: Number of input qubits
        output_value: The constant output (0 or 1)
    
    Returns:
        QuantumCircuit implementing the constant oracle
    
    Example:
        >>> oracle = create_constant_oracle(3, output_value=0)
        >>> # Creates f(x) = 0 for all 3-bit inputs
    """
    oracle = QuantumCircuit(n + 1, name=f"Const({output_value})")
    
    if output_value == 1:
        # f(x) = 1: Flip the ancilla qubit
        oracle.x(n)
    # f(x) = 0: Do nothing (identity on ancilla)
    
    return oracle


# Demonstrate constant oracles
print("=" * 50)
print("CONSTANT ORACLE: f(x) = 0 (no gates needed)")
print("=" * 50)
oracle_0 = create_constant_oracle(3, output_value=0)
print(oracle_0.draw())

print("\n" + "=" * 50)
print("CONSTANT ORACLE: f(x) = 1 (X gate on ancilla)")
print("=" * 50)
oracle_1 = create_constant_oracle(3, output_value=1)
print(oracle_1.draw())

### Understanding Constant Oracles

**For f(x) = 0**:
- The oracle is the identity operation
- $|x\rangle|y\rangle \rightarrow |x\rangle|y \oplus 0\rangle = |x\rangle|y\rangle$

**For f(x) = 1**:
- Apply X gate to ancilla: $|y\rangle \rightarrow |y \oplus 1\rangle$
- $|x\rangle|y\rangle \rightarrow |x\rangle|y \oplus 1\rangle$

## Section 4: Intermediate Implementation - Balanced Oracle

In [ ]:
def create_balanced_oracle(n: int, control_pattern: str = None) -> QuantumCircuit:
    """
    Create a balanced oracle: f(x) = s·x mod 2 (dot product with hidden string s).
    
    Args:
        n: Number of input qubits
        control_pattern: Binary string indicating which qubits control CNOT.
                        Must have at least one '1' for balanced function.
                        Default: "1" * n (all qubits control)
    
    Returns:
        QuantumCircuit implementing the balanced oracle
    
    Example:
        >>> oracle = create_balanced_oracle(3, "101")
        >>> # Creates f(x) = x_0 ⊕ x_2 (XOR of bits 0 and 2)
    """
    if control_pattern is None:
        control_pattern = "1" * n
    
    if len(control_pattern) != n:
        raise ValueError(f"control_pattern length {len(control_pattern)} != n ({n})")
    
    if "1" not in control_pattern:
        raise ValueError("control_pattern must have at least one '1' for balanced function")
    
    oracle = QuantumCircuit(n + 1, name=f"Bal({control_pattern})")
    
    # Apply CNOT from each qubit where pattern has '1'
    # Pattern is MSB to LSB, qubit indexing is LSB to MSB
    for i, bit in enumerate(reversed(control_pattern)):
        if bit == "1":
            oracle.cx(i, n)  # Control: qubit i, Target: ancilla (qubit n)
    
    return oracle


# Demonstrate balanced oracles with different patterns
patterns = ["111", "100", "011", "101"]

for pattern in patterns:
    print(f"\n{'=' * 50}")
    print(f"BALANCED ORACLE: control_pattern = '{pattern}'")
    print(f"Implements: f(x) = " + " ⊕ ".join([f"x_{i}" for i, b in enumerate(reversed(pattern)) if b == "1"]))
    print("=" * 50)
    oracle = create_balanced_oracle(3, pattern)
    print(oracle.draw())

### Verify Balanced Property

Let's verify that our balanced oracle really produces half 0s and half 1s:

In [ ]:
def verify_oracle_balance(oracle: QuantumCircuit, n: int) -> dict:
    """
    Verify if an oracle is balanced by checking all inputs classically.
    
    Args:
        oracle: The oracle circuit
        n: Number of input qubits
    
    Returns:
        Dictionary with verification results
    """
    outputs = {"0": 0, "1": 0}
    output_list = []
    
    for x in range(2**n):
        # Create circuit to test this input
        qc = QuantumCircuit(n + 1, 1)
        
        # Prepare input state |x⟩|0⟩
        binary_x = format(x, f'0{n}b')
        for i, bit in enumerate(reversed(binary_x)):
            if bit == '1':
                qc.x(i)
        
        # Apply oracle
        qc.compose(oracle, inplace=True)
        
        # Measure ancilla
        qc.measure(n, 0)
        
        # Run simulation
        simulator = AerSimulator()
        result = simulator.run(qc, shots=1).result()
        output = list(result.get_counts().keys())[0]
        
        outputs[output] += 1
        output_list.append((binary_x, output))
    
    is_constant = (outputs["0"] == 2**n) or (outputs["1"] == 2**n)
    is_balanced = (outputs["0"] == outputs["1"] == 2**(n-1))
    
    return {
        "outputs": outputs,
        "is_constant": is_constant,
        "is_balanced": is_balanced,
        "details": output_list
    }


# Verify a balanced oracle
n = 3
balanced_oracle = create_balanced_oracle(n, "101")
result = verify_oracle_balance(balanced_oracle, n)

print("Verification for balanced oracle (pattern '101'):")
print(f"  Outputs: {result['outputs']}")
print(f"  Is Constant: {result['is_constant']}")
print(f"  Is Balanced: {result['is_balanced']}")
print("\nTruth table:")
print("  Input -> Output")
for inp, out in result['details']:
    print(f"  |{inp}⟩ -> {out}")

## Section 5: Complete Deutsch-Jozsa Circuit

In [ ]:
def deutsch_jozsa_circuit(oracle: QuantumCircuit, n: int) -> QuantumCircuit:
    """
    Construct the complete Deutsch-Jozsa algorithm circuit.
    
    Args:
        oracle: The oracle circuit (must have n+1 qubits)
        n: Number of input qubits
    
    Returns:
        Complete DJ circuit with measurements
    
    Algorithm steps:
        1. Initialize: |0⟩^n |1⟩  (ancilla in |1⟩)
        2. Apply H^(n+1)
        3. Apply oracle U_f
        4. Apply H^n to input qubits
        5. Measure input qubits
    """
    # Create circuit with n+1 qubits and n classical bits
    qr = QuantumRegister(n, 'input')
    ancilla = QuantumRegister(1, 'ancilla')
    cr = ClassicalRegister(n, 'output')
    qc = QuantumCircuit(qr, ancilla, cr)
    
    # Step 1: Initialize ancilla to |1⟩
    qc.x(ancilla[0])
    
    # Step 2: Apply Hadamard to all qubits
    qc.h(qr)
    qc.h(ancilla[0])
    
    qc.barrier(label="init")
    
    # Step 3: Apply oracle
    qc.compose(oracle, inplace=True)
    
    qc.barrier(label="oracle")
    
    # Step 4: Apply Hadamard to input qubits only
    qc.h(qr)
    
    qc.barrier(label="final H")
    
    # Step 5: Measure input qubits
    qc.measure(qr, cr)
    
    return qc


# Create and display DJ circuit with constant oracle
n = 3
constant_oracle = create_constant_oracle(n, output_value=0)
dj_constant = deutsch_jozsa_circuit(constant_oracle, n)

print("Deutsch-Jozsa Circuit with CONSTANT Oracle (f=0):")
print(dj_constant.draw(fold=80))

In [ ]:
# Create and display DJ circuit with balanced oracle
balanced_oracle = create_balanced_oracle(n, "101")
dj_balanced = deutsch_jozsa_circuit(balanced_oracle, n)

print("Deutsch-Jozsa Circuit with BALANCED Oracle (f = x_0 ⊕ x_2):")
print(dj_balanced.draw(fold=80))

## Section 5.5: 📊 Step-by-Step State Evolution with Barriers

This section provides a **rigorous mathematical walkthrough** of how the quantum state evolves through each stage of the Deutsch-Jozsa algorithm. We use **barriers** to visually separate each stage and examine the state vector at each step.

### Circuit Stages with Barriers

```
         ┌───────────┐          ┌───────────┐         ┌───────────┐ ░ ┌─┐
         │  Stage 1  │          │  Stage 2  │         │  Stage 4  │ ░ │M│
         │Initialize │          │ Hadamard  │         │ Hadamard  │ ░ └╥┘
         └───────────┘          └───────────┘         └───────────┘ ░  ║
              |0⟩      ┌───┐          ┌────┐    ┌───┐              ░ ┌─┐
q_0: ─────────────────┤ H ├──║──────┤    ├──║──┤ H ├──────────────░─┤M├
                      ├───┤   ║      │    │   ║ ├───┤              ░ └╥┘
q_1: ─────────────────┤ H ├──║──────┤ Uf ├──║──┤ H ├──────────────░──║─
                      ├───┤   ║      │    │   ║ ├───┤              ░  ║
              |1⟩     ├───┤   ║      │    │   ║ └───┘              ░  ║
q_n: ────────X────────┤ H ├──║──────┤    ├──║─────────────────────░──║─
                      └───┘   ║      └────┘   ║                    ░  ║
       ⬆️State①    ⬆️State② ║  ⬆️State③    ║  ⬆️State④           ⬆️Meas
                          Barrier        Barrier
```

In [ ]:
def build_dj_circuit_with_barriers(oracle: QuantumCircuit, n: int, oracle_name: str = "Oracle"):
    """
    Build Deutsch-Jozsa circuit with explicit barriers at each stage
    for educational visualization of state evolution.
    
    Stages:
        Stage ① : Initialization (|0⟩^n ⊗ |1⟩)
        Stage ② : After Hadamards (uniform superposition ⊗ |−⟩)
        Stage ③ : After Oracle (phase kickback applied)
        Stage ④ : After final Hadamards (interference complete)
    """
    # Create quantum and classical registers
    qr_input = QuantumRegister(n, 'input')
    qr_ancilla = QuantumRegister(1, 'ancilla')
    cr = ClassicalRegister(n, 'measure')
    
    qc = QuantumCircuit(qr_input, qr_ancilla, cr, name=f"DJ-{oracle_name}")
    
    # ══════════════════════════════════════════════════════════════════
    # STAGE ① : INITIALIZATION
    # Prepare initial state: |0⟩^⊗n ⊗ |1⟩
    # Mathematical state: |ψ₁⟩ = |00...0⟩ ⊗ |1⟩
    # ══════════════════════════════════════════════════════════════════
    qc.x(qr_ancilla[0])  # Flip ancilla from |0⟩ to |1⟩
    
    qc.barrier(label="① Init: |0⟩ⁿ|1⟩")
    
    # ══════════════════════════════════════════════════════════════════
    # STAGE ② : APPLY HADAMARDS TO ALL QUBITS
    # Creates superposition of all inputs and |−⟩ state on ancilla
    # Mathematical state: |ψ₂⟩ = (1/√2ⁿ) Σₓ |x⟩ ⊗ |−⟩
    # where |−⟩ = (|0⟩ - |1⟩)/√2
    # ══════════════════════════════════════════════════════════════════
    qc.h(qr_input)       # H^⊗n on input qubits
    qc.h(qr_ancilla[0])  # H on ancilla: |1⟩ → |−⟩
    
    qc.barrier(label="② H⊗(n+1): superposition")
    
    # ══════════════════════════════════════════════════════════════════
    # STAGE ③ : APPLY ORACLE Uf (Phase Kickback)
    # Oracle transforms: |x⟩|−⟩ → (-1)^f(x) |x⟩|−⟩
    # Mathematical state: |ψ₃⟩ = (1/√2ⁿ) Σₓ (-1)^f(x) |x⟩ ⊗ |−⟩
    # The function value becomes a PHASE on the input register!
    # ══════════════════════════════════════════════════════════════════
    qc.compose(oracle, qubits=list(range(n+1)), inplace=True)
    
    qc.barrier(label="③ Oracle: phase kickback")
    
    # ══════════════════════════════════════════════════════════════════
    # STAGE ④ : APPLY HADAMARDS TO INPUT QUBITS (Interference)
    # Using H^⊗n |x⟩ = (1/√2ⁿ) Σz (-1)^(x·z) |z⟩
    # Mathematical state: |ψ₄⟩ = (1/2ⁿ) Σz [Σₓ (-1)^(f(x)+x·z)] |z⟩ ⊗ |−⟩
    # The amplitude of |0⟩^⊗n = (1/2ⁿ) Σₓ (-1)^f(x)
    # ══════════════════════════════════════════════════════════════════
    qc.h(qr_input)  # H^⊗n on input qubits only
    
    qc.barrier(label="④ Final H⊗n: interference")
    
    # ══════════════════════════════════════════════════════════════════
    # MEASUREMENT
    # Measure only the input qubits (not the ancilla)
    # If all zeros → CONSTANT function
    # If any non-zero → BALANCED function
    # ══════════════════════════════════════════════════════════════════
    qc.measure(qr_input, cr)
    
    return qc


# Build circuits with barriers for both oracle types
n = 3

# Constant oracle (f(x) = 0)
const_oracle = create_constant_oracle(n, 0)
dj_const_barriers = build_dj_circuit_with_barriers(const_oracle, n, "Constant")

# Balanced oracle (f(x) = x₀ ⊕ x₁ ⊕ x₂)
bal_oracle = create_balanced_oracle(n, "111")
dj_bal_barriers = build_dj_circuit_with_barriers(bal_oracle, n, "Balanced")

print("="*70)
print("DEUTSCH-JOZSA CIRCUIT WITH BARRIERS - CONSTANT ORACLE f(x) = 0")
print("="*70)
print(dj_const_barriers.draw(fold=-1))

print("\n" + "="*70)
print("DEUTSCH-JOZSA CIRCUIT WITH BARRIERS - BALANCED ORACLE f(x) = x₀⊕x₁⊕x₂")
print("="*70)
print(dj_bal_barriers.draw(fold=-1))

### 5.5.1 The Hadamard Identity and the "1-Matching" Rule

A key identity for understanding Deutsch-Jozsa is how Hadamard transforms any computational basis state:

$$H^{\otimes n}|a\rangle = \frac{1}{\sqrt{2^n}} \sum_{d=0}^{2^n-1} (-1)^{a \cdot d} |d\rangle$$

where $a \cdot d = a_n d_n + a_{n-1} d_{n-1} + \cdots + a_1 d_1 \pmod{2}$ (bitwise inner product)

**The "1-Matching" Rule for Signs:**
The sign $(-1)^{a \cdot d}$ is determined by counting positions where **BOTH** input bit $a_i$ AND output bit $d_i$ are 1:
- If count is **even** → sign is $+1$
- If count is **odd** → sign is $-1$

**Worked Example from Lecture** (5-bit):
```
Input  |a⟩ = |0 1 0 1 0⟩
Output |d⟩ = |1 0 0 1 0⟩
              ─────────────
Bit-AND:      0 0 0 1 0   ← Only position 4 has both bits = 1

Count = 1 (odd) → Sign = (-1)¹ = -1
```

This tells us that when $H^{\otimes 5}|01010\rangle$ is computed, the term $|10010\rangle$ appears with a **negative** sign.

In [ ]:
def one_matching_sign(input_bits: str, output_bits: str) -> int:
    """
    Compute the sign using the 1-matching rule from the lecture.
    
    Count positions where BOTH input AND output bits are 1.
    If count is even → +1, if odd → -1
    """
    assert len(input_bits) == len(output_bits), "Bit strings must have same length"
    matches = sum(1 for a, d in zip(input_bits, output_bits) if a == '1' and d == '1')
    return (-1) ** matches


# Verify the lecture example: |01010⟩ → |10010⟩
print("Lecture Example: H⊗⁵|01010⟩ → coefficient of |10010⟩")
print("=" * 50)
sign = one_matching_sign("01010", "10010")
print(f"Input:  |01010⟩")
print(f"Output: |10010⟩")
print(f"1-matches: position 4 only (0∧1=0, 1∧0=0, 0∧0=0, 1∧1=1, 0∧0=0)")
print(f"Count = 1 (odd) → Sign = {sign}")
print(f"\nH⊗⁵|01010⟩ contains: {'+' if sign > 0 else '-'}1/√32 × |10010⟩")

### 5.5.2 State Evolution at Each Barrier

Let's trace through the DJ algorithm showing the quantum state at each barrier point.

**State ①** (After X on ancilla): $|\psi_1\rangle = |0\rangle^{\otimes n} \otimes |1\rangle$

**State ②** (After Hadamards): 
$$|\psi_2\rangle = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} |x\rangle \otimes |{-}\rangle$$
- Input register: uniform superposition (all signs positive since $0 \cdot x = 0$)
- Ancilla: $H|1\rangle = |{-}\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$

**State ③** (After Oracle - Phase Kickback):
$$|\psi_3\rangle = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} (-1)^{f(x)} |x\rangle \otimes |{-}\rangle$$
- The function value $f(x)$ becomes a **phase** on each $|x\rangle$!

**State ④** (After Final Hadamards - Interference):
$$|\psi_4\rangle = \frac{1}{2^n} \sum_{s} \left[\sum_{d} (-1)^{f(d) + d \cdot s}\right] |s\rangle \otimes |{-}\rangle$$

**Amplitude of $|0\rangle^{\otimes n}$**: Since $d \cdot 0 = 0$ for all $d$:
$$\alpha_{|0\rangle^n} = \frac{1}{2^n} \sum_{d=0}^{2^n-1} (-1)^{f(d)}$$

| Function | Sum | Amplitude | Probability |
|----------|-----|-----------|-------------|
| **Constant** | $\pm 2^n$ | $\pm 1$ | **100%** |
| **Balanced** | $0$ | $0$ | **0%** |

In [ ]:
def trace_dj_evolution(n_qubits: int = 2, oracle_type: str = 'constant_0'):
    """
    Trace state evolution through DJ algorithm using 1-matching rule.
    
    Parameters:
        n_qubits: Number of input qubits
        oracle_type: 'constant_0', 'constant_1', or 'balanced'
    """
    print(f"="*60)
    print(f"Deutsch-Jozsa State Evolution ({n_qubits} qubits, {oracle_type})")
    print(f"="*60)
    
    # Build circuit
    qc, stages = build_dj_circuit_with_barriers(n_qubits, oracle_type)
    
    # Get oracle function values
    if oracle_type == 'constant_0':
        f = lambda x: 0
    elif oracle_type == 'constant_1':
        f = lambda x: 1
    else:  # balanced
        f = lambda x: bin(x).count('1') % 2  # XOR-like balanced function
    
    N = 2**n_qubits
    
    # ═══════════════ State ② After Initial Hadamards ═══════════════
    print(f"\n📍 STATE ② (After Hadamards on all qubits)")
    print(f"   Input register: uniform superposition over {N} states")
    print(f"   |ψ₂⟩ = (1/√{N}) Σ |x⟩ ⊗ |−⟩")
    print(f"\n   All amplitudes: +1/√{N} (since 0·x = 0 → even 1-matches)")
    
    # ═══════════════ State ③ After Oracle ═══════════════
    print(f"\n📍 STATE ③ (After Oracle - Phase Kickback)")
    print(f"   |ψ₃⟩ = (1/√{N}) Σ (-1)^f(x) |x⟩ ⊗ |−⟩")
    print(f"\n   Function values encoded as phases:")
    
    phases = []
    for x in range(N):
        fx = f(x)
        sign = '+' if fx == 0 else '-'
        phases.append((x, fx, sign))
        x_bin = format(x, f'0{n_qubits}b')
        print(f"      |{x_bin}⟩: f({x_bin})={fx} → sign = {sign}1")
    
    # ═══════════════ State ④ After Final Hadamards ═══════════════
    print(f"\n📍 STATE ④ (After Final Hadamards - Interference)")
    print(f"   Using 1-matching rule: H⊗ⁿ|d⟩ = (1/√{N}) Σₛ (-1)^(d·s) |s⟩")
    
    # Calculate amplitude of |0⟩^n
    sum_phases = sum((-1)**f(d) for d in range(N))
    amplitude = sum_phases / N
    
    print(f"\n   Amplitude of |{'0'*n_qubits}⟩:")
    print(f"   α = (1/{N}) × Σ_d (-1)^f(d)")
    print(f"     = (1/{N}) × ({sum_phases})")
    print(f"     = {amplitude}")
    
    prob_zero = abs(amplitude)**2
    print(f"\n   P(|{'0'*n_qubits}⟩) = |α|² = {prob_zero:.4f} = {prob_zero*100:.1f}%")
    
    # ═══════════════ Final Result ═══════════════
    print(f"\n" + "═"*60)
    if oracle_type.startswith('constant'):
        print(f"✓ CONSTANT function: All {N} phases same → constructive interference")
        print(f"  Measuring |{'0'*n_qubits}⟩ with 100% probability")
    else:
        print(f"✓ BALANCED function: Half +1, half -1 → destructive interference")
        print(f"  |{'0'*n_qubits}⟩ amplitude = 0, never measured")
    print("═"*60)
    
    return qc

# Run traces for different oracle types
print("▶ CONSTANT ORACLE (f(x) = 0 for all x)")
print()
# Create the oracle first, then pass it to build_dj_circuit_with_barriers
oracle_const = create_constant_oracle(2, output_value=0)
qc_const = build_dj_circuit_with_barriers(oracle_const, 2, "Constant-0")

# Now manually trace through since trace_dj_evolution doesn't exist yet
print("="*60)
print("Deutsch-Jozsa State Evolution (2 qubits, constant_0)")
print("="*60)
print("\nCircuit structure created. For full state tracing,")
print("the trace_dj_evolution function needs to be implemented.")

print("\n\n")
print("▶ BALANCED ORACLE (f(x) = parity of x)")
print()
# Create balanced oracle first
oracle_bal = create_balanced_oracle(2, "11")  # parity function for 2 qubits
qc_bal = build_dj_circuit_with_barriers(oracle_bal, 2, "Balanced")

# Manual trace for balanced oracle
print("="*60)
print("Deutsch-Jozsa State Evolution (2 qubits, balanced)")
print("="*60)

N = 4  # 2^2
print(f"\n📍 STATE ② (After Hadamards on all qubits)")
print(f"   Input register: uniform superposition over {N} states")
print(f"   |ψ₂⟩ = (1/√{N}) Σ |x⟩ ⊗ |−⟩")

print(f"\n📍 STATE ③ (After Oracle - Phase Kickback)")
print(f"   |ψ₃⟩ = (1/√{N}) Σ (-1)^f(x) |x⟩ ⊗ |−⟩")
print(f"   Function values (parity): f(00)=0, f(01)=1, f(10)=1, f(11)=0")

print(f"\n📍 STATE ④ (After Final Hadamards - Interference)")
sum_phases = 1 - 1 - 1 + 1  # (-1)^0 + (-1)^1 + (-1)^1 + (-1)^0
amplitude = sum_phases / N
print(f"   Amplitude of |00⟩ = (1/{N}) × ({sum_phases}) = {amplitude}")
print(f"   P(|00⟩) = {abs(amplitude)**2:.4f}")

print(f"\n" + "="*60)
print(f"✓ BALANCED function: Destructive interference at |00⟩")
print("="*60)

### 5.5.3 The 1-Matching Rule in Action

Let's verify the 1-matching rule with a concrete example from the lecture:

**Example**: For $|01010\rangle$ (input $a$) in the Hadamard expansion:
$$H^{\otimes 5}|01010\rangle = \frac{1}{\sqrt{32}} \sum_{d=0}^{31} (-1)^{01010 \cdot d} |d\rangle$$

What's the sign of $|10010\rangle$ (output $d$) in this expansion?

| Position | $a$ bit | $d$ bit | Both = 1? |
|----------|---------|---------|-----------|
| 0 | 0 | 0 | ❌ |
| 1 | 1 | 1 | ✅ |
| 2 | 0 | 0 | ❌ |
| 3 | 1 | 0 | ❌ |
| 4 | 0 | 0 | ❌ |

**Result**: 1 match (odd) → sign = $(-1)^1 = -1$

In [ ]:
# Interactive 1-matching calculator
def explore_1_matching(input_a: str, output_d: str):
    """
    Explore the 1-matching rule for determining Hadamard expansion signs.
    
    From the lecture: When we expand H⊗ⁿ|a⟩, the sign of |d⟩ is determined by
    counting positions where BOTH a AND d have a 1 (the "1-matching" rule).
    """
    if len(input_a) != len(output_d):
        print("❌ Input and output must have same length!")
        return
    
    n = len(input_a)
    print(f"Hadamard expansion: H⊗{n}|{input_a}⟩")
    print(f"Finding sign of |{output_d}⟩ in this expansion...")
    print()
    
    # Create table
    print("Position | a bit | d bit | Both=1?")
    print("-" * 40)
    
    matches = 0
    for i, (a_bit, d_bit) in enumerate(zip(input_a, output_d)):
        both = a_bit == '1' and d_bit == '1'
        if both:
            matches += 1
        status = "✅" if both else "❌"
        print(f"   {i}     |   {a_bit}   |   {d_bit}   |   {status}")
    
    print("-" * 40)
    parity = "even" if matches % 2 == 0 else "odd"
    sign = (-1) ** matches
    sign_str = "+" if sign == 1 else "-"
    
    print(f"\n1-matches: {matches} ({parity})")
    print(f"Sign: (-1)^{matches} = {sign_str}1")
    print(f"\n✓ In H⊗{n}|{input_a}⟩, the amplitude of |{output_d}⟩ is {sign_str}1/√{2**n}")

# Lecture example: |01010⟩ → |10010⟩
print("="*50)
print("LECTURE EXAMPLE")
print("="*50)
explore_1_matching("01010", "10010")

print("\n")
print("="*50)
print("TRY IT YOURSELF")
print("="*50)
explore_1_matching("11", "10")  # 2-qubit example

### 5.5.4 📊 Summary: Key Takeaways

| Stage | Mathematical State | Physical Meaning |
|-------|-------------------|------------------|
| **① Init** | $|0\rangle^{\otimes n} \otimes |1\rangle$ | Start with ancilla in $|1\rangle$ |
| **② After H** | $\frac{1}{\sqrt{2^n}} \sum_x |x\rangle \otimes |{-}\rangle$ | Uniform superposition (all signs +1 via 1-matching: $0 \cdot x = 0$) |
| **③ After Oracle** | $\frac{1}{\sqrt{2^n}} \sum_x (-1)^{f(x)} |x\rangle \otimes |{-}\rangle$ | **Phase kickback**: $f(x)$ encoded in phases |
| **④ After Final H** | Use 1-matching rule for each amplitude | **Interference** concentrates amplitude |

**The 1-Matching Rule**: Sign of $|d\rangle$ in $H^{\otimes n}|a\rangle$ = $(-1)^{\text{count of positions where both } a_i=1 \text{ and } d_i=1}$

**The Magic**: 
- Constant $f$: All $(-1)^{f(x)}$ same → **constructive** interference at $|0\rangle^{\otimes n}$
- Balanced $f$: Half $+1$, half $-1$ → **destructive** interference at $|0\rangle^{\otimes n}$

**Quantum Advantage**: Classically need $2^{n-1}+1$ queries. Quantum needs exactly **1**!

## Section 6: Visualization & State Evolution

In [ ]:
def visualize_dj_evolution(oracle: QuantumCircuit, n: int, oracle_name: str):
    """
    Visualize the quantum state at each step of Deutsch-Jozsa.
    """
    print(f"\n{'=' * 60}")
    print(f"STATE EVOLUTION: {oracle_name}")
    print("=" * 60)
    
    # Build circuit step by step
    qc = QuantumCircuit(n + 1)
    
    # Step 0: Initial state
    state = Statevector(qc)
    print("\nStep 0 - Initial |0...0⟩:")
    print(f"  State: {state}")
    
    # Step 1: X on ancilla
    qc.x(n)
    state = Statevector(qc)
    print("\nStep 1 - After X on ancilla |0...0⟩|1⟩:")
    print(f"  State: {state}")
    
    # Step 2: Hadamard on all
    qc.h(range(n + 1))
    state = Statevector(qc)
    print("\nStep 2 - After H^(n+1) (superposition):")
    print(f"  Non-zero amplitudes: {len([a for a in state.data if abs(a) > 1e-10])}")
    print(f"  First few amplitudes: {state.data[:4]}...")
    
    # Step 3: Oracle
    qc.compose(oracle, inplace=True)
    state = Statevector(qc)
    print("\nStep 3 - After Oracle:")
    print(f"  First few amplitudes: {state.data[:4]}...")
    
    # Step 4: Hadamard on input qubits
    qc.h(range(n))
    state = Statevector(qc)
    print("\nStep 4 - After final H^n:")
    print(f"  Probabilities (first 8 states):")
    probs = state.probabilities()
    for i in range(min(8, len(probs))):
        if probs[i] > 1e-10:
            print(f"    |{format(i, f'0{n+1}b')}⟩: {probs[i]:.4f}")
    
    return state


# Visualize evolution for constant oracle
const_oracle = create_constant_oracle(3, 0)
state_const = visualize_dj_evolution(const_oracle, 3, "CONSTANT Oracle (f=0)")

# Visualize evolution for balanced oracle
bal_oracle = create_balanced_oracle(3, "111")
state_bal = visualize_dj_evolution(bal_oracle, 3, "BALANCED Oracle (f=x_0⊕x_1⊕x_2)")

In [ ]:
# Plot probability distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Constant oracle results
probs_const = state_const.probabilities()
# Group by input qubit measurement (trace out ancilla)
input_probs_const = np.zeros(2**3)
for i in range(2**4):
    input_idx = i >> 1  # Remove ancilla bit
    input_probs_const[input_idx] += probs_const[i]

axes[0].bar(range(8), input_probs_const, color='blue', alpha=0.7)
axes[0].set_xlabel('Input Measurement Outcome')
axes[0].set_ylabel('Probability')
axes[0].set_title('CONSTANT Oracle\n(f(x) = 0)')
axes[0].set_xticks(range(8))
axes[0].set_xticklabels([format(i, '03b') for i in range(8)])
axes[0].axhline(y=1.0, color='r', linestyle='--', label='P(000) = 1')
axes[0].legend()

# Balanced oracle results
probs_bal = state_bal.probabilities()
input_probs_bal = np.zeros(2**3)
for i in range(2**4):
    input_idx = i >> 1
    input_probs_bal[input_idx] += probs_bal[i]

axes[1].bar(range(8), input_probs_bal, color='orange', alpha=0.7)
axes[1].set_xlabel('Input Measurement Outcome')
axes[1].set_ylabel('Probability')
axes[1].set_title('BALANCED Oracle\n(f(x) = x₀⊕x₁⊕x₂)')
axes[1].set_xticks(range(8))
axes[1].set_xticklabels([format(i, '03b') for i in range(8)])
axes[1].axhline(y=0.0, color='r', linestyle='--', label='P(000) = 0')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n📊 Key Observation:")
print("  • CONSTANT oracle: P(000) = 1.00 → Guaranteed to measure |000⟩")
print("  • BALANCED oracle: P(000) = 0.00 → Will NEVER measure |000⟩")

## Section 7: Common Traps Demonstration

In [ ]:
# TRAP 1: Forgetting to initialize ancilla to |1⟩
print("=" * 60)
print("TRAP 1: Forgetting to initialize ancilla to |1⟩")
print("=" * 60)

def deutsch_jozsa_BUGGY_no_x(oracle: QuantumCircuit, n: int) -> QuantumCircuit:
    """BUGGY VERSION: Forgets X gate on ancilla"""
    qc = QuantumCircuit(n + 1, n)
    # BUG: Missing qc.x(n) here!
    qc.h(range(n + 1))
    qc.barrier()
    qc.compose(oracle, inplace=True)
    qc.barrier()
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

# Test buggy version with balanced oracle
n = 3
balanced_oracle = create_balanced_oracle(n, "111")
buggy_circuit = deutsch_jozsa_BUGGY_no_x(balanced_oracle, n)

simulator = AerSimulator()
result = simulator.run(buggy_circuit, shots=1000).result()
counts = result.get_counts()

print("\nBuggy circuit result (should detect BALANCED, but...):")
print(f"  Measurement outcomes: {counts}")
if '000' in counts:
    print(f"  ❌ BUG: Got |000⟩ with probability {counts['000']/1000:.2%}")
    print("  This would incorrectly suggest CONSTANT function!")
print("\n  ✓ Fix: Add qc.x(n) before the Hadamards to prepare |1⟩ on ancilla")

In [ ]:
# TRAP 2: Measuring the ancilla qubit
print("\n" + "=" * 60)
print("TRAP 2: Measuring the ancilla qubit")
print("=" * 60)

def deutsch_jozsa_BUGGY_measure_all(oracle: QuantumCircuit, n: int) -> QuantumCircuit:
    """BUGGY VERSION: Measures all qubits including ancilla"""
    qc = QuantumCircuit(n + 1, n + 1)  # BUG: n+1 classical bits
    qc.x(n)
    qc.h(range(n + 1))
    qc.compose(oracle, inplace=True)
    qc.h(range(n))
    qc.measure(range(n + 1), range(n + 1))  # BUG: Measuring ancilla too
    return qc

buggy_circuit_2 = deutsch_jozsa_BUGGY_measure_all(balanced_oracle, n)
result = simulator.run(buggy_circuit_2, shots=1000).result()
counts = result.get_counts()

print("\nBuggy circuit result (measuring ancilla):")
print(f"  Measurement outcomes: {counts}")
print("\n  The output is now 4-bit strings (includes ancilla)!")
print("  ✓ Fix: Only measure the n input qubits, not the ancilla.")
print("  The ancilla is a 'catalyst' - its state is always |-⟩.")

In [ ]:
# TRAP 3: Using a function that's neither constant nor balanced
print("\n" + "=" * 60)
print("TRAP 3: Function that's neither constant nor balanced")
print("=" * 60)

# Create a "broken" oracle that only has CNOT on one qubit out of 3
# This is NOT balanced for n=3 (only 4 outputs are 1, not 4)
# Wait, actually CNOT from q0 to ancilla IS balanced!
# Let's create a truly non-balanced oracle

def create_invalid_oracle(n: int) -> QuantumCircuit:
    """Create an oracle that's neither constant nor balanced."""
    oracle = QuantumCircuit(n + 1, name="INVALID")
    # Use CCX (Toffoli) - only flips when both q0 and q1 are 1
    # For n=3: only inputs 011, 111 give output 1
    # That's 2 outputs of 1, 6 outputs of 0 -> NOT balanced!
    oracle.ccx(0, 1, n)
    return oracle

invalid_oracle = create_invalid_oracle(3)
print("\nInvalid oracle (Toffoli gate):")
print(invalid_oracle.draw())

# Verify it's not balanced
result = verify_oracle_balance(invalid_oracle, 3)
print(f"\nOracle verification:")
print(f"  Outputs: {result['outputs']}")
print(f"  Is Constant: {result['is_constant']}")
print(f"  Is Balanced: {result['is_balanced']}")
print("  ⚠️  This oracle is NEITHER constant NOR balanced!")

# Run DJ on invalid oracle
dj_invalid = deutsch_jozsa_circuit(invalid_oracle, 3)
result = simulator.run(dj_invalid, shots=1000).result()
counts = result.get_counts()

print(f"\nDeutsch-Jozsa result on INVALID oracle:")
print(f"  Outcomes: {counts}")
print("\n  ⚠️  The result is MEANINGLESS because the promise is violated!")
print("  DJ algorithm assumes f is constant OR balanced - nothing else.")

## Section 8: Simulator Experiments

In [ ]:
def run_deutsch_jozsa_experiment(n: int, num_trials: int = 10):
    """
    Run comprehensive DJ experiment with various oracles.
    """
    simulator = AerSimulator()
    results = []
    
    # Test constant oracles
    for output in [0, 1]:
        oracle = create_constant_oracle(n, output)
        circuit = deutsch_jozsa_circuit(oracle, n)
        
        job = simulator.run(circuit, shots=1)
        counts = job.result().get_counts()
        measurement = list(counts.keys())[0]
        
        detected = "CONSTANT" if measurement == "0" * n else "BALANCED"
        correct = detected == "CONSTANT"
        
        results.append({
            "type": "CONSTANT",
            "output": output,
            "measurement": measurement,
            "detected": detected,
            "correct": correct
        })
    
    # Test balanced oracles with various patterns
    import random
    for _ in range(num_trials):
        # Generate random non-zero pattern
        pattern = "".join([str(random.randint(0, 1)) for _ in range(n)])
        while pattern == "0" * n:
            pattern = "".join([str(random.randint(0, 1)) for _ in range(n)])
        
        oracle = create_balanced_oracle(n, pattern)
        circuit = deutsch_jozsa_circuit(oracle, n)
        
        job = simulator.run(circuit, shots=1)
        counts = job.result().get_counts()
        measurement = list(counts.keys())[0]
        
        detected = "CONSTANT" if measurement == "0" * n else "BALANCED"
        correct = detected == "BALANCED"
        
        results.append({
            "type": "BALANCED",
            "pattern": pattern,
            "measurement": measurement,
            "detected": detected,
            "correct": correct
        })
    
    return results


# Run experiment
n = 4
results = run_deutsch_jozsa_experiment(n, num_trials=20)

print(f"Deutsch-Jozsa Experiment Results (n={n})")
print("=" * 70)

# Summarize
constant_results = [r for r in results if r["type"] == "CONSTANT"]
balanced_results = [r for r in results if r["type"] == "BALANCED"]

print("\nCONSTANT Oracles:")
for r in constant_results:
    status = "✓" if r["correct"] else "✗"
    print(f"  {status} f(x)={r['output']}: measured {r['measurement']} → detected {r['detected']}")

print("\nBALANCED Oracles (sample):")
for r in balanced_results[:5]:
    status = "✓" if r["correct"] else "✗"
    print(f"  {status} pattern={r['pattern']}: measured {r['measurement']} → detected {r['detected']}")

total_correct = sum(1 for r in results if r["correct"])
print(f"\n📊 Summary: {total_correct}/{len(results)} correct ({100*total_correct/len(results):.0f}%)")
print("   Expected: 100% (quantum algorithm is deterministic)")

## Section 9: Hardware Considerations (Conceptual)

### Running on Real Hardware

When running Deutsch-Jozsa on real quantum hardware, several factors affect results:

1. **Gate Errors**: Each gate has ~0.1-1% error rate
2. **Decoherence**: Qubits lose quantum properties over time
3. **Measurement Errors**: ~1-5% error in reading qubit states
4. **Connectivity**: Not all qubits can interact directly

In [ ]:
# Simulate hardware noise effects
from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error

def create_noisy_simulator(gate_error_rate: float = 0.01):
    """
    Create a simulator with depolarizing noise to mimic real hardware.
    """
    noise_model = NoiseModel()
    
    # Add depolarizing error to single-qubit gates
    error_1q = depolarizing_error(gate_error_rate, 1)
    noise_model.add_all_qubit_quantum_error(error_1q, ['h', 'x'])
    
    # Add depolarizing error to two-qubit gates (usually higher)
    error_2q = depolarizing_error(gate_error_rate * 5, 2)
    noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])
    
    return AerSimulator(noise_model=noise_model)


# Compare ideal vs noisy results
n = 3
shots = 1024

balanced_oracle = create_balanced_oracle(n, "111")
dj_circuit = deutsch_jozsa_circuit(balanced_oracle, n)

# Ideal simulator
ideal_sim = AerSimulator()
ideal_result = ideal_sim.run(dj_circuit, shots=shots).result()
ideal_counts = ideal_result.get_counts()

# Noisy simulator
noisy_sim = create_noisy_simulator(gate_error_rate=0.02)
noisy_result = noisy_sim.run(dj_circuit, shots=shots).result()
noisy_counts = noisy_result.get_counts()

print("Comparison: Ideal vs Noisy Simulation")
print("=" * 50)
print(f"Oracle: Balanced (pattern '111')")
print(f"Shots: {shots}")
print()
print("IDEAL Simulator:")
print(f"  Results: {ideal_counts}")
print(f"  P(000) = {ideal_counts.get('000', 0)/shots:.4f}")
print()
print("NOISY Simulator (2% gate error):")
print(f"  Results: {noisy_counts}")
print(f"  P(000) = {noisy_counts.get('000', 0)/shots:.4f}")
print()
print("💡 Note: On noisy hardware, you may see small probability for |000⟩")
print("   This is due to gate errors, not a flaw in the algorithm.")

In [ ]:
# Visualize noise impact
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Ideal results
labels = [format(i, '03b') for i in range(8)]
ideal_probs = [ideal_counts.get(l, 0)/shots for l in labels]
axes[0].bar(labels, ideal_probs, color='blue', alpha=0.7)
axes[0].set_xlabel('Measurement Outcome')
axes[0].set_ylabel('Probability')
axes[0].set_title('IDEAL Simulator')
axes[0].set_ylim(0, 1.1)

# Noisy results
noisy_probs = [noisy_counts.get(l, 0)/shots for l in labels]
axes[1].bar(labels, noisy_probs, color='red', alpha=0.7)
axes[1].set_xlabel('Measurement Outcome')
axes[1].set_ylabel('Probability')
axes[1].set_title('NOISY Simulator (2% gate error)')
axes[1].set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

## Section 10: Exercises

### Exercise 1: Implement and Verify (Beginner)
Create a Deutsch-Jozsa circuit for n=2 with a constant oracle (f=1). 
Verify that the output is always |00⟩.

In [ ]:
# TODO: Exercise 1
# 1. Create a constant oracle with n=2, output=1
# 2. Build the DJ circuit
# 3. Run on simulator
# 4. Verify output is |00⟩

# Your code here:
# oracle = ...
# circuit = ...
# result = ...
# print(f"Result: {result.get_counts()}")

### Exercise 2: Custom Balanced Oracle (Intermediate)
Create a balanced oracle that implements f(x) = x₁ ⊕ x₃ for n=4.
Verify it's balanced and that DJ correctly identifies it.

In [ ]:
# TODO: Exercise 2
# 1. Determine the correct control_pattern for f(x) = x₁ ⊕ x₃
# 2. Create the balanced oracle
# 3. Verify it's balanced using verify_oracle_balance()
# 4. Run DJ and confirm it detects "balanced"

# Your code here:

### Exercise 3: Noise Analysis (Advanced)
Investigate how error rate affects DJ accuracy. Plot accuracy vs. error rate for error rates from 0% to 10%.

In [ ]:
# TODO: Exercise 3
# 1. Create a function that tests DJ accuracy for a given error rate
# 2. Test multiple error rates: 0%, 1%, 2%, 5%, 10%
# 3. Plot accuracy vs error rate
# 4. Discuss: At what error rate does DJ become unreliable?

# Your code here:

## Section 11: Quick Knowledge Check

Test your understanding with these questions:

**Q1**: What is the purpose of the ancilla qubit in Deutsch-Jozsa?

<details>
<summary>Click for answer</summary>
The ancilla enables phase kickback. When initialized to |−⟩, the oracle's action |y⟩ → |y⊕f(x)⟩ becomes a phase: |x⟩|−⟩ → (−1)^f(x)|x⟩|−⟩. This converts function values into phases that can interfere.
</details>

**Q2**: Why does DJ need exactly one query while classical deterministic needs 2^(n-1)+1?

<details>
<summary>Click for answer</summary>
Quantum superposition lets us query all 2^n inputs simultaneously. Interference then amplifies/cancels paths to reveal the global property (constant vs balanced) in the measurement outcome. Classical algorithms must check inputs sequentially.
</details>

**Q3**: Why doesn't DJ provide a "practical" speedup?

<details>
<summary>Click for answer</summary>
A randomized classical algorithm can solve this with O(1) queries and high probability: just check a few random inputs. If they're all the same, the function is almost certainly constant. DJ's speedup is only vs. deterministic classical algorithms.
</details>

## Section 12: Summary & Next Steps

### Key Takeaways

1. **Deutsch-Jozsa** is the first quantum algorithm to show exponential speedup
2. **Phase kickback** converts function values to phases via the ancilla
3. **Quantum interference** amplifies the answer (000 for constant, other for balanced)
4. **The promise** (constant OR balanced) is essential—algorithm fails without it
5. **Practical relevance**: Techniques (not the algorithm itself) are foundational

### What's Next?

- **Module 7.2**: Bernstein-Vazirani algorithm (same circuit, different oracle interpretation)
- **Module 7.3**: Simon's algorithm (exponential speedup for period finding)
- **Module 7.4**: Grover's algorithm (quadratic speedup for unstructured search)

### Further Exploration

- Try running on IBM Quantum hardware via `qiskit-ibm-runtime`
- Implement other balanced oracles (not just XOR-based)
- Explore the relationship between DJ and Bernstein-Vazirani